In [1]:
import json
import joblib
from pprint import pprint
from pathlib import Path

import pandas as pd
import numpy as np

from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer

from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline

import xgboost as xgb
from xgboost.callback import EarlyStopping

from isic_helper import DotDict
from isic_helper import get_folds
from isic_helper import compute_auc, compute_pauc

import matplotlib.pyplot as plt

pd.options.display.max_columns = 1000

In [2]:
cfg = DotDict()
cfg.seed = 2022

cfg.models_output_dir = "models"
cfg.model_name = "xgb_v1"

cfg.sampling_ratio = 0.01

In [3]:
id_column = 'isic_id'
target_column = 'target'
group_column = 'patient_id'

numerical_features = [
    "age_approx",
    "clin_size_long_diam_mm",
    "tbp_lv_A", "tbp_lv_Aext",
    "tbp_lv_B", "tbp_lv_Bext",
    "tbp_lv_C", "tbp_lv_Cext",
    "tbp_lv_H", "tbp_lv_Hext",
    "tbp_lv_L", "tbp_lv_Lext",
    "tbp_lv_areaMM2",
    "tbp_lv_area_perim_ratio",
    "tbp_lv_color_std_mean",
    "tbp_lv_deltaA", "tbp_lv_deltaB", "tbp_lv_deltaL", "tbp_lv_deltaLB", "tbp_lv_deltaLBnorm",
    "tbp_lv_eccentricity",
    "tbp_lv_minorAxisMM",
    "tbp_lv_nevi_confidence",
    "tbp_lv_norm_border", "tbp_lv_norm_color",
    "tbp_lv_perimeterMM",
    "tbp_lv_radial_color_std_max",
    "tbp_lv_stdL", "tbp_lv_stdLExt",
    "tbp_lv_symm_2axis", "tbp_lv_symm_2axis_angle",
    "tbp_lv_x", "tbp_lv_y", "tbp_lv_z",
]

ord_categorical_features = []

ohe_categorical_features = [
    "sex", 
    "anatom_site_general", 
    "tbp_tile_type",
    "tbp_lv_location", 
    "tbp_lv_location_simple",
    "attribution",
]

def preprocess(df):
    df["anatom_site_general"] = df["anatom_site_general"].fillna("missing")
    df["sex"] = df["sex"].fillna("missing")
    df["tbp_tile_type"] = df["tbp_tile_type"].map({"3D: white": "white", "3D: XP": "XP"})
    df["anatom_site_general"] = df["anatom_site_general"].str.replace(" ", "-").str.replace("/", "OR")
    df["tbp_lv_location"] = df["tbp_lv_location"].str.replace(" ", "").str.replace("&", "AND")
    df["tbp_lv_location_simple"] = df["tbp_lv_location_simple"].str.replace(" ", "").str.replace("&", "AND")
    attribution_mapper = {
        "Memorial Sloan Kettering Cancer Center": "MSKCC",
        "ACEMID MIA": "ACEMIDMIA",
        "Department of Dermatology, Hospital Clínic de Barcelona": "DoD_HCB",
        "University Hospital of Basel": "UHB",
        "Frazer Institute, The University of Queensland, Dermatology Research Centre": "FI_TUQ-DRC",
        "Department of Dermatology, University of Athens, Andreas Syggros Hospital of Skin and Venereal Diseases, Alexander Stratigos, Konstantinos Liopyris": "DoD_UA",
        "ViDIR Group, Department of Dermatology, Medical University of Vienna": "ViDIR"
    }
    df["attribution"] = df["attribution"].map(attribution_mapper)
    
    return df

def norm_feature(df, value_col, group_cols=[group_column], err=1e-5):
    stats = ["mean", "std"]
    tmp = df.groupby(group_cols)[value_col].agg(stats)
    tmp.columns = [f"{value_col}_{stat}" for stat in stats]
    tmp.reset_index(inplace=True)
    df = df.merge(tmp, on=group_cols, how="left")
    feature_name = f"{value_col}_patient_norm"
    df[feature_name] = ((df[value_col] - df[f"{value_col}_mean"]) / 
                                       (df[f"{value_col}_std"] + err))
    return df, feature_name

def count_features(df, col):
    tmp = df[[id_column, group_column, col]].pivot_table(
        values=id_column, 
        index=group_column, 
        columns=col, 
        aggfunc="count", 
        fill_value=0)
    feature_cols = tmp.columns.tolist()
    tmp.reset_index(inplace=True)
    tmp.index.name = None
    df = df.merge(tmp, on=group_column, how="left")
    return df, feature_cols

def feature_engineering(df, err=1e-5):
    new_num_cols = []
    
    df["lesion_size_ratio"] = df["tbp_lv_minorAxisMM"] / df["clin_size_long_diam_mm"]
    new_num_cols += ["lesion_size_ratio"]
    
    df["lesion_shape_index"] = df["tbp_lv_areaMM2"] / df["tbp_lv_perimeterMM"]**2
    new_num_cols += ["lesion_shape_index"]
    
    df["hue_contrast"] = np.abs(df["tbp_lv_H"] - df["tbp_lv_Hext"])
    new_num_cols += ["hue_contrast"]
    
    df["luminance_contrast"] = np.abs(df["tbp_lv_L"] - df["tbp_lv_Lext"])
    new_num_cols += ["luminance_contrast"]
    
    df["lesion_color_difference"] = np.sqrt(df["tbp_lv_deltaA"]**2 +
                                            df["tbp_lv_deltaB"]**2 +
                                            df["tbp_lv_deltaL"]**2)
    new_num_cols += ["lesion_color_difference"]
    
    df["border_complexity"] = df["tbp_lv_norm_border"] + df["tbp_lv_symm_2axis"]
    new_num_cols += ["border_complexity"]
    
    df["color_uniformity"] = df["tbp_lv_color_std_mean"] / (df["tbp_lv_radial_color_std_max"] + err)
    new_num_cols += ["color_uniformity"]
    
    df["position_distance_3d"] = np.sqrt(df["tbp_lv_x"]**2 +
                                         df["tbp_lv_y"]**2 +
                                         df["tbp_lv_z"]**2)
    new_num_cols += ["position_distance_3d"]
    
    df["perimeter_to_area_ratio"] = df["tbp_lv_perimeterMM"] / df["tbp_lv_areaMM2"]
    new_num_cols += ["perimeter_to_area_ratio"]
    
    df["area_to_perimeter_ratio"] = df["tbp_lv_areaMM2"] / df["tbp_lv_perimeterMM"]
    new_num_cols += ["area_to_perimeter_ratio"]
    
    df["lesion_visibility_score"] = df["tbp_lv_deltaLBnorm"] + df["tbp_lv_norm_color"]
    new_num_cols += ["lesion_visibility_score"]
    
    df["symmetry_border_consistency"] = df["tbp_lv_symm_2axis"] * df["tbp_lv_norm_border"]
    new_num_cols += ["symmetry_border_consistency"]
    
    df["consistency_symmetry_border"] = (df["tbp_lv_symm_2axis"] * df["tbp_lv_norm_border"] /
                                         (df["tbp_lv_symm_2axis"] + df["tbp_lv_norm_border"]))
    new_num_cols += ["consistency_symmetry_border"]
    
    df["color_consistency"] = df["tbp_lv_stdL"] / df["tbp_lv_Lext"]
    new_num_cols += ["color_consistency"]
    
    df["consistency_color"] = (df["tbp_lv_stdL"] * df["tbp_lv_Lext"] /
                               (df["tbp_lv_stdL"] * df["tbp_lv_Lext"]))
    new_num_cols += ["consistency_color"]
    
    df["size_age_interaction"] = df["clin_size_long_diam_mm"] * df["age_approx"]
    new_num_cols += ["size_age_interaction"]
    
    df["hue_color_std_interaction"] = df["tbp_lv_H"] * df["tbp_lv_color_std_mean"]
    new_num_cols += ["hue_color_std_interaction"]
    
    df["lesion_severity_index"] = (df["tbp_lv_norm_border"] +
                                   df["tbp_lv_norm_color"] +
                                   df["tbp_lv_eccentricity"]) / 3
    new_num_cols += ["lesion_severity_index"]
    
    df["shape_complexity_index"] = df["border_complexity"] + df["lesion_shape_index"]
    new_num_cols += ["shape_complexity_index"]
    
    df["color_contrast_index"] = (df["tbp_lv_deltaA"] +
                                  df["tbp_lv_deltaB"] + 
                                  df["tbp_lv_deltaL"] +
                                  df["tbp_lv_deltaLBnorm"])
    new_num_cols += ["color_contrast_index"]
    
    df["log_lesion_area"] = np.log1p(df["tbp_lv_areaMM2"])
    new_num_cols += ["log_lesion_area"]
    
    df["normalized_lesion_size"] = df["clin_size_long_diam_mm"] / df["age_approx"]
    new_num_cols += ["normalized_lesion_size"]
    
    df["mean_hue_difference"] = (df["tbp_lv_H"] + df["tbp_lv_Hext"]) / 2
    new_num_cols += ["mean_hue_difference"]
    
    df["std_dev_contrast"] = np.sqrt((df["tbp_lv_deltaA"]**2 +
                                      df["tbp_lv_deltaB"]**2 + 
                                      df["tbp_lv_deltaL"]**2) / 3)
    new_num_cols += ["std_dev_contrast"]
    
    df["color_shape_composite_index"] = (df["tbp_lv_color_std_mean"] + 
                                         df["tbp_lv_area_perim_ratio"] +
                                         df["tbp_lv_symm_2axis"]) / 3
    new_num_cols += ["color_shape_composite_index"]
    
    df["lesion_orientation_3d"] = np.arctan2(df["tbp_lv_y"], df["tbp_lv_x"])
    new_num_cols += ["lesion_orientation_3d"]
    
    df["overall_color_difference"] = (df["tbp_lv_deltaA"] + 
                                      df["tbp_lv_deltaB"] + 
                                      df["tbp_lv_deltaL"]) / 3
    new_num_cols += ["overall_color_difference"]
    
    df["symmetry_perimeter_interaction"] = df["tbp_lv_symm_2axis"] * df["tbp_lv_perimeterMM"]
    new_num_cols += ["symmetry_perimeter_interaction"]
    
    df["comprehensive_lesion_index"] = (df["tbp_lv_area_perim_ratio"] +
                                        df["tbp_lv_eccentricity"] +
                                        df["tbp_lv_norm_color"] +
                                        df["tbp_lv_symm_2axis"]) / 4
    new_num_cols += ["comprehensive_lesion_index"]
    
    df["color_variance_ratio"] = df["tbp_lv_color_std_mean"] / df["tbp_lv_stdLExt"]
    new_num_cols += ["color_variance_ratio"]
    
    df["border_color_interaction"] = df["tbp_lv_norm_border"] * df["tbp_lv_norm_color"]
    new_num_cols += ["border_color_interaction"]
    
    df["border_color_interaction_2"] = ((df["tbp_lv_norm_border"] * df["tbp_lv_norm_color"]) /
                                        (df["tbp_lv_norm_border"] + df["tbp_lv_norm_color"]))
    new_num_cols += ["border_color_interaction_2"]
    
    df["size_color_contrast_ratio"] = df["clin_size_long_diam_mm"] / df["tbp_lv_deltaLBnorm"]
    new_num_cols += ["size_color_contrast_ratio"]
    
    df["age_normalized_nevi_confidence"] = df["tbp_lv_nevi_confidence"] / df["age_approx"]
    new_num_cols += ["age_normalized_nevi_confidence"]
    
    df["age_normalized_nevi_confidence_2"] = np.sqrt(df["tbp_lv_nevi_confidence"]**2 + df["age_approx"]**2)
    new_num_cols += ["age_normalized_nevi_confidence_2"]
    
    df["color_asymmetry_index"] = df["tbp_lv_radial_color_std_max"] * df["tbp_lv_symm_2axis"]
    new_num_cols += ["color_asymmetry_index"]
    
    df["volume_approximation_3d"] = df["tbp_lv_areaMM2"] * np.sqrt(df["tbp_lv_x"]**2 +
                                                                   df["tbp_lv_y"]**2 +
                                                                   df["tbp_lv_z"]**2)
    new_num_cols += ["volume_approximation_3d"]
    
    df["color_range"] = (np.abs(df["tbp_lv_L"] - df["tbp_lv_Lext"]) +
                         np.abs(df["tbp_lv_A"] - df["tbp_lv_Aext"]) +
                         np.abs(df["tbp_lv_B"] - df["tbp_lv_Bext"]))
    new_num_cols += ["color_range"]
    
    df["shape_color_consistency"] = df["tbp_lv_eccentricity"] * df["tbp_lv_color_std_mean"]
    new_num_cols += ["shape_color_consistency"]
    
    df["border_length_ratio"] = df["tbp_lv_perimeterMM"] / np.sqrt(2 * df["tbp_lv_areaMM2"])
    new_num_cols += ["border_length_ratio"]
    
    df["age_size_symmetry_index"] = (df["age_approx"] *
                                     df["clin_size_long_diam_mm"] *
                                     df["tbp_lv_symm_2axis"])
    new_num_cols += ["age_size_symmetry_index"]
    
    df["age_area_symmetry"] = (df["age_approx"] *
                               df["tbp_lv_areaMM2"] *
                               df["tbp_lv_symm_2axis"])
    new_num_cols += ["age_area_symmetry"]
    
    for col in numerical_features + new_num_cols:
        df, feature_name = norm_feature(df, col, group_cols=["tbp_tile_type", "anatom_site_general"])
        new_num_cols += [feature_name]
        
    df, feature_cols = count_features(df, "anatom_site_general")
    new_num_cols += feature_cols
    
    df["num_images"] = df.patient_id.map(df.groupby(group_column)[id_column].count())
    new_num_cols += ["num_images"]

    return df, new_num_cols

In [4]:
INPUT_PATH = Path("../input/isic-2024-challenge/")
MODELS_OUTPUT_PATH = Path(f"{cfg.models_output_dir}")
MODELS_OUTPUT_PATH.mkdir(exist_ok=True)

train_metadata = pd.read_csv(INPUT_PATH / "train-metadata.csv", low_memory=False, na_values=["NA"])
test_metadata = pd.read_csv(INPUT_PATH / "test-metadata.csv", low_memory=False, na_values=["NA"])

folds_df = get_folds()
train_metadata = train_metadata.merge(folds_df, on=["isic_id", "patient_id"], how="inner")
print(f"Train data size: {train_metadata.shape}")
print(f"Test data size: {test_metadata.shape}")

train_metadata = preprocess(train_metadata)
test_metadata = preprocess(test_metadata)

train_metadata, new_num_cols = feature_engineering(train_metadata)
test_metadata, _ = feature_engineering(test_metadata)

numerical_features += new_num_cols

Train data size: (401059, 57)
Test data size: (3, 44)


In [5]:
mixed_encoded_preprocessor = ColumnTransformer(
    [
        ("numerical", "passthrough", numerical_features),
        (
            "ord_categorical",
            OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-2, encoded_missing_value=-1, dtype=int),
            ord_categorical_features,
        ),
        (
            "ohe_categorical",
            OneHotEncoder(sparse_output=False, dtype=np.int32, handle_unknown="ignore"),
            ohe_categorical_features
        )
    ],
    verbose_feature_names_out=False,

)
mixed_encoded_preprocessor.set_output(transform="pandas")

with open(f"{cfg.model_name}_encoder.joblib", "wb") as f:
    joblib.dump(mixed_encoded_preprocessor, f)

In [6]:
train_ids = train_metadata[id_column]
groups = train_metadata[group_column]
folds = train_metadata["fold"]

enc = mixed_encoded_preprocessor.fit(train_metadata)
X_train = enc.transform(train_metadata)
y_train = train_metadata[target_column]

categorical_features = ord_categorical_features[:]
for ohe_col in ohe_categorical_features:
    categorical_features += [col for col in X_train.columns if col.startswith(ohe_col)]

print(f"Total number of columns: {len(X_train.columns)}")

Total number of columns: 206


In [7]:
X_train.head()

age_approx  clin_size_long_diam_mm   tbp_lv_A  tbp_lv_Aext   tbp_lv_B  \
0        60.0                    3.04  20.244422    16.261975  26.922447   
1        60.0                    1.10  31.712570    25.364740  26.331000   
2        60.0                    3.40  22.575830    17.128170  37.970460   
3        65.0                    3.22  14.242329    12.164757  21.448144   
4        55.0                    2.73  24.725520    20.057470  26.464900   

   tbp_lv_Bext   tbp_lv_C  tbp_lv_Cext   tbp_lv_H  tbp_lv_Hext   tbp_lv_L  \
0    23.954773  33.684638    28.953117  53.058545    55.828924  54.367448   
1    24.549290  41.219030    35.299260  39.702910    44.064040  48.861520   
2    33.485410  44.174920    37.611800  59.265850    62.909730  53.961180   
3    21.121356  25.746200    24.374023  56.414429    60.060388  18.649518   
4    25.710460  36.217980    32.608740  46.946070    52.041180  46.276310   

   tbp_lv_Lext  tbp_lv_areaMM2  tbp_lv_area_perim_ratio  \
0    62.025701        3.152561                27.476170   
1    55.362360        0.919497                12.235290   
2    61.670520        3.265153                24.184620   
3    23.314841        6.079940                14.889242   
4    54.855740        2.101708                19.902560   

   tbp_lv_color_std_mean  tbp_lv_deltaA  tbp_lv_deltaB  tbp_lv_deltaL  \
0                0.00000       3.982447       2.967674      -7.658253   
1                0.00000       6.347830       1.781713      -6.500838   
2                0.00000       5.447655       4.485044      -7.709336   
3                0.51452       2.077572       0.326788      -4.665323   
4                0.00000       4.668053       0.754434      -8.579431   

   tbp_lv_deltaLB  tbp_lv_deltaLBnorm  tbp_lv_eccentricity  \
0        8.360566            5.784302             0.901302   
1        6.839008            4.987244             0.639885   
2        9.092376            6.290359             0.932147   
3        4.783413            6.400196             0.654458   
4        9.148495            6.531302             0.946448   

   tbp_lv_minorAxisMM  tbp_lv_nevi_confidence  tbp_lv_norm_border  \
0            1.543016            2.628592e-03            7.091360   
1            0.821918            1.334303e-07            2.116402   
2            1.194905            2.959177e-04            4.798335   
3            2.481328            2.198945e+01            1.975874   
4            0.929916            1.378832e-03            3.658854   

   tbp_lv_norm_color  tbp_lv_perimeterMM  tbp_lv_radial_color_std_max  \
0           0.000000            9.307003                      0.00000   
1           0.000000            3.354148                      0.00000   
2           0.000000            8.886309                      0.00000   
3           1.771705            9.514499                      0.66469   
4           0.000000            6.467562                      0.00000   

   tbp_lv_stdL  tbp_lv_stdLExt  tbp_lv_symm_2axis  tbp_lv_symm_2axis_angle  \
0     2.036195        2.637780           0.590476                       85   
1     0.853227        3.912844           0.285714                       55   
2     1.743651        1.950777           0.361905                      105   
3     1.258541        1.573733           0.209581                      130   
4     2.085409        2.480509           0.313433                       20   

     tbp_lv_x     tbp_lv_y    tbp_lv_z  lesion_size_ratio  lesion_shape_index  \
0 -182.703552   613.493652  -42.427948           0.507571            0.036395   
1   -0.078308  1575.687000   57.174500           0.747198            0.081731   
2  123.649700  1472.010000  232.908900           0.351443            0.041349   
3 -141.024780  1442.185791   58.359802           0.770599            0.067163   
4  -72.315640  1488.720000   21.428960           0.340629            0.050245   

   hue_contrast  luminance_contrast  lesion_color_difference  \
0      2.770379            7.658253       

In [8]:
def pauc_80(y_train, y_pred):
    score_value = compute_pauc(y_train, y_pred, min_tpr=0.8)   
    return score_value


num_rounds = 2000
es_rounds = 150
log_rounds = 50

params = {
    'enable_categorical': True,
    'tree_method':        'hist',
    'random_state':       cfg.seed,
    'learning_rate':      0.08501257473292347, 
    'lambda':             8.879624125465703, 
    'alpha':              0.6779926606782505, 
    'max_depth':          6, 
    'subsample':          0.6012681388711075, 
    'colsample_bytree':   0.8437772277074493, 
    'colsample_bylevel':  0.5476090898823716, 
    'colsample_bynode':   0.9928601203635129, 
    'scale_pos_weight':   3.29440313334688,
    'verbosity':          0,
    'disable_default_eval_metric': True
}

In [9]:
best_num_rounds = {}
val_auc_scores = {}
val_pauc_scores = {}
all_folds = np.sort(folds.unique())
oof_predictions = np.zeros(X_train.shape[0])
for fold in all_folds:
    print(f"Running fold: {fold}")
    dev_index = folds[folds != fold].index
    val_index = folds[folds == fold].index
    
    X_dev = X_train.loc[dev_index, :]
    y_dev = y_train[dev_index]
    
    X_val = X_train.loc[val_index, :]
    y_val = y_train[val_index]
    
    model = Pipeline([
        ('sampler', RandomUnderSampler(sampling_strategy=cfg.sampling_ratio, random_state=cfg.seed)),
        ('classifier', xgb.XGBClassifier(n_estimators=num_rounds, **params, eval_metric=pauc_80, 
                                         callbacks=[EarlyStopping(rounds=es_rounds, metric_name='pauc_80', maximize=True, save_best=True)])),
    ])
    model.fit(X_dev, y_dev,
              classifier__eval_set=[(X_val, y_val)],
              classifier__verbose=log_rounds
             )
    best_num_rounds[f"fold_{fold}"] = model._final_estimator.best_iteration
    
    val_preds = model.predict_proba(X_val)[:, -1]
    oof_predictions[val_index] = val_preds
    
    val_auc_scores[f"fold_{fold}"] = compute_auc(y_val, val_preds)
    val_pauc_scores[f"fold_{fold}"] = compute_pauc(y_val, val_preds, min_tpr=0.8)
    print("\n")
    
    with open(MODELS_OUTPUT_PATH / f"{cfg.model_name}_fold_{fold}.txt", "wb") as f:
        joblib.dump(model, f)

print("Val AUC scores:")
pprint(val_auc_scores)
print("Val PAUC scores:")
pprint(val_pauc_scores)

oof_preds_df = pd.DataFrame({
    id_column: train_ids,
    group_column: groups,
    "fold": folds,
    target_column: y_train,
    f"oof_{cfg.model_name}": oof_predictions
})
oof_preds_df.to_csv(f"oof_preds_{cfg.model_name}.csv", index=False)

cv_auc_oof = compute_auc(oof_preds_df[target_column], oof_preds_df[f"oof_{cfg.model_name}"])
cv_pauc_oof = compute_pauc(oof_preds_df[target_column], oof_preds_df[f"oof_{cfg.model_name}"], min_tpr=0.8)

cv_auc_avg = np.mean(list(val_auc_scores.values()))
cv_pauc_avg = np.mean(list(val_pauc_scores.values()))

cv_auc_std = np.std(list(val_auc_scores.values()))
cv_pauc_std = np.std(list(val_pauc_scores.values()))

print(f"CV AUC OOF: {cv_auc_oof}")
print(f"CV PAUC OOF: {cv_pauc_oof}")
print(f"CV AUC AVG: {cv_auc_avg}")
print(f"CV PAUC AVG: {cv_pauc_avg}")
print(f"CV AUC STD: {cv_auc_std}")
print(f"CV PAUC STD: {cv_pauc_std}")
print(X_train.columns)

Running fold: 1
[0]	validation_0-pauc_80:0.06519
[50]	validation_0-pauc_80:0.16403
[100]	validation_0-pauc_80:0.16409
[150]	validation_0-pauc_80:0.16698
[200]	validation_0-pauc_80:0.16630
[250]	validation_0-pauc_80:0.16649
[300]	validation_0-pauc_80:0.16626
[326]	validation_0-pauc_80:0.16570


Running fold: 2
[0]	validation_0-pauc_80:0.07597
[50]	validation_0-pauc_80:0.16005
[100]	validation_0-pauc_80:0.16300
[150]	validation_0-pauc_80:0.16368
[200]	validation_0-pauc_80:0.16447
[250]	validation_0-pauc_80:0.16401
[300]	validation_0-pauc_80:0.16406
[350]	validation_0-pauc_80:0.16472
[400]	validation_0-pauc_80:0.16528
[450]	validation_0-pauc_80:0.16534
[500]	validation_0-pauc_80:0.16603
[550]	validation_0-pauc_80:0.16626
[600]	validation_0-pauc_80:0.16600
[650]	validation_0-pauc_80:0.16646
[700]	validation_0-pauc_80:0.16608
[750]	validation_0-pauc_80:0.16669
[800]	validation_0-pauc_80:0.16617
[850]	validation_0-pauc_80:0.16633
[900]	validation_0-pauc_80:0.16590


Running fold: 3
[0]	valid

In [10]:
feature_importances = 0
for fold in all_folds:
    model_filepath = MODELS_OUTPUT_PATH / f"{cfg.model_name}_fold_{fold}.txt"
    with open(model_filepath, "rb") as f:
        model = joblib.load(f)
    if fold == 1:
        feature_names = model._final_estimator.feature_names_in_
    feature_importances += model._final_estimator.feature_importances_
feature_importances = feature_importances.astype(np.float32)
feature_importances /= len(all_folds)

In [11]:
imp = pd.DataFrame.from_dict(dict(zip(feature_names, feature_importances)), 
                             orient="index", columns=["imp"]).sort_values("imp", ascending=True)
imp.plot(kind="barh", figsize=(10, 25))
plt.show() 

In [12]:
metadata = {
    "params": params,
    "num_rounds": num_rounds,
    "es_rounds": es_rounds,
    "best_num_rounds": best_num_rounds,
    "val_auc_scores": val_auc_scores,
    "val_pauc_scores": val_pauc_scores,
    "cv_auc_oof": cv_auc_oof,
    "cv_pauc_oof": cv_pauc_oof,
    "cv_auc_avg": cv_auc_avg,
    "cv_pauc_avg": cv_pauc_avg
}

with open(f"{cfg.model_name}_run_metadata.json", "w") as f:
    json.dump(metadata, f)